### Testing First Model : StableDiffusionLDM3DPipeline
from paper : https://arxiv.org/abs/2305.10853

In [2]:
from diffusers import StableDiffusionLDM3DPipeline

/DATA/ai22resch11001/anaconda3/envs/cs_work/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
pipe = StableDiffusionLDM3DPipeline.from_pretrained("Intel/ldm3d-pano")
pipe.to("cuda")

diffusion_pytorch_model.safetensors: 100%|██████████| 3.44G/3.44G [02:54<00:00, 19.7MB/s]
Loading pipeline components...:  14%|█▍        | 1/7 [00:00<00:00,  6.04it/s]`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.
Loading pipeline components...: 100%|██████████| 7/7 [00:00<00:00, 12.07it/s]


StableDiffusionLDM3DPipeline {
  "_class_name": "StableDiffusionLDM3DPipeline",
  "_diffusers_version": "0.25.1",
  "_name_or_path": "Intel/ldm3d-pano",
  "feature_extractor": [
    "transformers",
    "CLIPImageProcessor"
  ],
  "image_encoder": [
    null,
    null
  ],
  "requires_safety_checker": true,
  "safety_checker": [
    "stable_diffusion",
    "StableDiffusionSafetyChecker"
  ],
  "scheduler": [
    "diffusers",
    "DDIMScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [34]:
prompts_set1 = [
    "360-degree panoramic image, of a simple room",
    "360-degree panoramic image, of a luxury room",
    "360-degree panoramic image, of a spaceship",
    "360-degree panoramic image, of a forest",
    "360-degree panoramic image, of an anime style natural landscape"
]

prompts_set2 = [
    "360-degree panoramic image, of a simple room, stereoscopic, equirectangular",
    "360-degree panoramic image, of a luxury room, stereoscopic, equirectangular",
    "360-degree panoramic image, of a spaceship, stereoscopic, equirectangular",
    "360-degree panoramic image, of a forest, stereoscopic, equirectangular",
    "360-degree panoramic image, of an anime style natural landscape, stereoscopic, equirectangular"
]


In [5]:
output = pipe(
        prompts_set1,
        width=1024,
        height=512,
        guidance_scale=5.0,
        num_inference_steps=50,
    ) 


100%|██████████| 50/50 [01:22<00:00,  1.65s/it]


In [8]:
rgb_image, depth_image = output.rgb, output.depth

In [39]:
from PIL import Image

def save_images_with_prompt_prefix(dest_pth,model_name, images, prompts):
    
    if not os.path.exists(dest_pth):
        os.mkdir(dest_pth)

    for i, image in enumerate(images):
        prompt_name = prompts[i].replace(" ", "_")  # Replace spaces with underscores
        filename = dest_pth + f"{model_name}_{prompt_name}_image_{i + 1}.png"
        image.save(filename)
        print(f"Image {i + 1} saved as {filename}")

In [19]:
save_images_with_prompt_prefix(rgb_image, prompts_set1)


Image 1 saved as LDM_360-degree_panoramic_image,_of_a_simple_room_image_1.png
Image 2 saved as LDM_360-degree_panoramic_image,_of_a_luxury_room_image_2.png
Image 3 saved as LDM_360-degree_panoramic_image,_of_a_spaceship_image_3.png
Image 4 saved as LDM_360-degree_panoramic_image,_of_a_forest_image_4.png
Image 5 saved as LDM_360-degree_panoramic_image,_of_an_anime_style_natural_landscape_image_5.png


In [20]:
output2 = pipe(
        prompts_set2,
        width=1024,
        height=512,
        guidance_scale=5.0,
        num_inference_steps=50,
    ) 

100%|██████████| 50/50 [01:22<00:00,  1.64s/it]


In [22]:
rgb_image, depth_image = output2.rgb, output2.depth

In [23]:
save_images_with_prompt_prefix('/DATA/ai22resch11001/temp_work/assets/LDM3D_pipeline_prompt_2/', 'LDM', rgb_image, prompts_set2)

Image 1 saved as /DATA/ai22resch11001/temp_work/assets/LDM3D_pipeline_prompt_2/LDM_360-degree_panoramic_image,_of_a_simple_room,_stereoscopic,_equirectangular_image_1.png
Image 2 saved as /DATA/ai22resch11001/temp_work/assets/LDM3D_pipeline_prompt_2/LDM_360-degree_panoramic_image,_of_a_luxury_room,_stereoscopic,_equirectangular_image_2.png
Image 3 saved as /DATA/ai22resch11001/temp_work/assets/LDM3D_pipeline_prompt_2/LDM_360-degree_panoramic_image,_of_a_spaceship,_stereoscopic,_equirectangular_image_3.png
Image 4 saved as /DATA/ai22resch11001/temp_work/assets/LDM3D_pipeline_prompt_2/LDM_360-degree_panoramic_image,_of_a_forest,_stereoscopic,_equirectangular_image_4.png
Image 5 saved as /DATA/ai22resch11001/temp_work/assets/LDM3D_pipeline_prompt_2/LDM_360-degree_panoramic_image,_of_an_anime_style_natural_landscape,_stereoscopic,_equirectangular_image_5.png


### MODEL 2 : Stable_DiffusionPanoramaPipeline
from paper : https://huggingface.co/papers/2302.08113

multi-diffusion which introduces interesting idea that we can have seperate image space(panorama) from a image space(normal picture).

In [24]:
import torch
from diffusers import StableDiffusionPanoramaPipeline, DDIMScheduler


In [28]:

model_ckpt = "stabilityai/stable-diffusion-2-base"
scheduler = DDIMScheduler.from_pretrained(model_ckpt, subfolder="scheduler")
pipe = StableDiffusionPanoramaPipeline.from_pretrained(model_ckpt, scheduler=scheduler, torch_dtype=torch.float16)
pipe = pipe.to("cuda")


Loading pipeline components...: 100%|██████████| 6/6 [00:01<00:00,  4.98it/s]


In [30]:

images = pipe(prompts_set1, guidance_scale=5, width=1024, height=512,
    circular_padding=True).images

100%|██████████| 50/50 [02:28<00:00,  2.96s/it]


In [33]:
save_images_with_prompt_prefix('/DATA/ai22resch11001/temp_work/assets/MultiDiffusion/', 'MultiDiffusion', images, prompts_set1)

Image 1 saved as /DATA/ai22resch11001/temp_work/assets/MultiDiffusion/MultiDiffusion_360-degree_panoramic_image,_of_a_simple_room_image_1.png
Image 2 saved as /DATA/ai22resch11001/temp_work/assets/MultiDiffusion/MultiDiffusion_360-degree_panoramic_image,_of_a_luxury_room_image_2.png
Image 3 saved as /DATA/ai22resch11001/temp_work/assets/MultiDiffusion/MultiDiffusion_360-degree_panoramic_image,_of_a_spaceship_image_3.png
Image 4 saved as /DATA/ai22resch11001/temp_work/assets/MultiDiffusion/MultiDiffusion_360-degree_panoramic_image,_of_a_forest_image_4.png
Image 5 saved as /DATA/ai22resch11001/temp_work/assets/MultiDiffusion/MultiDiffusion_360-degree_panoramic_image,_of_an_anime_style_natural_landscape_image_5.png


In [34]:

images2 = pipe(prompts_set2, guidance_scale=5, width=1024, height=512,
    circular_padding=True).images

100%|██████████| 50/50 [02:42<00:00,  3.25s/it]


In [40]:
save_images_with_prompt_prefix('/DATA/ai22resch11001/temp_work/assets/MultiDiffusion_prompt_2/', 'MultiDiffusion', images2, prompts_set2)

Image 1 saved as /DATA/ai22resch11001/temp_work/assets/MultiDiffusion_prompt_2/MultiDiffusion_360-degree_panoramic_image,_of_a_simple_room,_stereoscopic,_equirectangular_image_1.png
Image 2 saved as /DATA/ai22resch11001/temp_work/assets/MultiDiffusion_prompt_2/MultiDiffusion_360-degree_panoramic_image,_of_a_luxury_room,_stereoscopic,_equirectangular_image_2.png
Image 3 saved as /DATA/ai22resch11001/temp_work/assets/MultiDiffusion_prompt_2/MultiDiffusion_360-degree_panoramic_image,_of_a_spaceship,_stereoscopic,_equirectangular_image_3.png
Image 4 saved as /DATA/ai22resch11001/temp_work/assets/MultiDiffusion_prompt_2/MultiDiffusion_360-degree_panoramic_image,_of_a_forest,_stereoscopic,_equirectangular_image_4.png
Image 5 saved as /DATA/ai22resch11001/temp_work/assets/MultiDiffusion_prompt_2/MultiDiffusion_360-degree_panoramic_image,_of_an_anime_style_natural_landscape,_stereoscopic,_equirectangular_image_5.png


#### TEMP CODE FOR RENAMING! ---------------


In [3]:
prompts_set1

['360-degree panoramic image, of a simple room',
 '360-degree panoramic image, of a luxury room',
 '360-degree panoramic image, of a spaceship',
 '360-degree panoramic image, of a forest',
 '360-degree panoramic image, of an anime style natural landscape']

In [8]:
import os

image_folder = "/DATA/ai22resch11001/temp_work/Results"  # Replace with the path to your image folder
prompt_set1 = [
    "360-degree panoramic image, of a simple room",
    "360-degree panoramic image, of a luxury room",
    "360-degree panoramic image, of a spaceship",
    "360-degree panoramic image, of a forest",
    "360-degree panoramic image, of an anime style natural landscape"
]

for idx, filename in enumerate(os.listdir(image_folder)):
    if filename.endswith(".png"):
        # Extract the timestamp and index from the filename
        parts = filename.split("_")
        timestamp, index = parts[1], parts[3]
        print(timestamp, index)
        # Choose the corresponding prompt from prompt_set1 based on the index
        prompt = prompt_set1[idx]
        # Create the new filename with the appended prompt
        new_filename = f"im_{timestamp}_{index}_{prompt}.png"

        # Rename the file
        os.rename(os.path.join(image_folder, filename), os.path.join(image_folder, new_filename))


20240118103233 11.png
20240118103749 11.png
20240118105318 11.png
20240118104813 11.png
20240118104301 11.png


In [31]:
for idx, filename in enumerate(os.listdir(image_folder)):
    print(idx, filename)
    print(prompt_set1[idx])
    print('-----')

0 im_20240118104813_11.png_360-degree panoramic image, of a forest.png
360-degree panoramic image, of a simple room
-----
1 im_20240118103749_11.png_360-degree panoramic image, of a luxury room.png
360-degree panoramic image, of a luxury room
-----
2 im_20240118103233_11.png_360-degree panoramic image, of a simple room.png
360-degree panoramic image, of a spaceship
-----
3 im_20240118105318_11.png_360-degree panoramic image, of a spaceship.png
360-degree panoramic image, of a forest
-----
4 im_20240118104301_11.png_360-degree panoramic image, of an anime style natural landscape.png
360-degree panoramic image, of an anime style natural landscape
-----
